---

In [1]:
import pandas as pd
import numpy as np
import tweepy
import requests as rq
import json
from yelp.client import Client
from tweepy.parsers import JSONParser
from geopy.geocoders import Nominatim
geolocator = Nominatim()

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

consumer_key = "Ed4RNulN1lp7AbOooHa9STCoU"
consumer_secret = "P7cUJlmJZq0VaCY0Jg7COliwQqzK0qYEyUF9Y0idx4ujb3ZlW5"
access_token = "839621358724198402-dzdOsx2WWHrSuBwyNUiqSEnTivHozAZ"
access_token_secret = "dCZ80uNRbFDjxdU2EckmNiSckdoATach6Q8zb7YYYE5ER"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=JSONParser())

target_cities = [
                 "Chicago",
                 "New York",
                 "Houston",
                 "Philadelphia",
                 "Los Angeles",
                 "Atlanta",
                 "Miami",
                 ]

In [2]:
coordinates = []
for city in target_cities:
    location = geolocator.geocode(city)
    coordinates.append({"Latitude": location.latitude,
                        "Longitude": location.longitude,
                        "City": city})
    
coordinates_df = pd.DataFrame.from_dict(coordinates)
coordinates_df = coordinates_df.set_index("City", inplace = False)

In [3]:
coordinates_df.to_csv("city_coordinates_dataframe.csv")

In [4]:
coordinates_data = pd.read_csv("city_coordinates_dataframe.csv")



for index, row in coordinates_data.iterrows():
    geo_code = ('%s,%s,20mi' %(row["Latitude"],row["Longitude"]))
    
    
    geo_coordinates = []
    geo_coordinates.append(geo_code)
        

    print(geo_coordinates)
    
    sentiment_df = pd.read_csv("LA-restaurant.csv")
    
    compound_list = []

    for index, row in sentiment_df.iterrows():
        public_tweets = api.search(row["User Name"], rpp=100, result_type="recent", geocode = geo_coordinates[0])   
  
        

        for tweet in public_tweets["statuses"]:
            for element in target_cities:
                compound = analyzer.polarity_scores(tweet["text"])["compound"]
                compound_list.append(compound)
                sentiment_df.set_value(index, element, np.mean(compound_list))
            

['41.8755546,-87.6244212,20mi']
['40.7306458,-73.9866136,20mi']
['29.7589382,-95.36769740000001,20mi']
['39.952399299999996,-75.1635899,20mi']
['34.054935,-118.244476,20mi']
['33.749098700000005,-84.39018490000001,20mi']
['25.7742658,-80.1936589,20mi']


In [11]:
sentiment_df = sentiment_df.dropna()
del sentiment_df['Unnamed: 0']

In [12]:
sentiment_df

,Restaurant Names,User Name,Follower Count,Chicago,New York,Houston,Philadelphia,Los Angeles,Atlanta,Miami
0,Red Lobster,redlobster,296664,0.396564,0.394859,0.393194,0.391567,0.389976,0.388421,0.386900
1,SUBWAY®Restaurants,SUBWAY,2467427,0.213683,0.211380,0.209100,0.206844,0.204611,0.202402,0.200214
3,In-N-Out Burger,innoutburger,44775,0.218667,0.220409,0.222136,0.223848,0.225545,0.227227,0.228894
6,Little Caesars Pizza,littlecaesars,232112,0.198930,0.198147,0.197370,0.196599,0.195834,0.195075,0.194322
8,Chili's Grill & Bar,Chilis,383907,0.198746,0.198182,0.197620,0.197062,0.196507,0.195955,0.195406
9,Fatburger,Fatburger,14993,0.210512,0.210741,0.210969,0.211195,0.211421,0.211645,0.211869
10,Wingstop,wingstop,208253,0.163071,0.162729,0.162390,0.162051,0.161715,0.161379,0.161045
11,Chuck E. Cheese's,ChuckECheeses,23749,0.163424,0.164005,0.164583,0.165160,0.165734,0.166306,0.166875
12,TGI Fridays,TGIFridays,208950,0.151398,0.151126,0.150854,0.150583,0.150313,0.150044,0.149776
14,Benihana,Benihana,13086,0.136453,0.136247,0.136040,0.135835,0.135630,0.135426,0.135222


In [13]:
sentiment_df.to_csv("city_sentiments.csv")